In [ ]:
!pip install transformers
!pip install huggingface_hub
!pip install datasets
!pip install evaluate
# !pip install torchtext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 30.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
!pip install spacy
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 82.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Personal Transformer
import spacy
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
raw_datasets = load_dataset("ashwinradhe/dl_dataset")
small_train_dataset = raw_datasets['train'].select(range(100000))
SRC_LANGUAGE = "en"
TRG_LANGUAGE = "fr"
tokenizer_en = get_tokenizer('spacy', language='en_core_web_sm')
tokenizer_fr= get_tokenizer('spacy', language='fr_core_news_sm')
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']
def tokenize_and_build_vocab(data, tokenizer):
    tokenized_data = [tokenizer(sentence) for sentence in data]
    vocab = build_vocab_from_iterator(tokenized_data,min_freq=1, specials=special_symbols,special_first=True)
    vocab.set_default_index(vocab['<unk>'])
    return vocab, tokenized_data
vocab_en, tokenized_data_en = tokenize_and_build_vocab(small_train_dataset['English Sentence'], tokenizer_en)
vocab_fr, tokenized_data_fr = tokenize_and_build_vocab(small_train_dataset['French Sentence'], tokenizer_fr)
def numericalize_and_pad(tokenized_data, vocab):
    numericalized_data = [torch.tensor(vocab.lookup_indices(sentence)) for sentence in tokenized_data]
    padded_data = pad_sequence(numericalized_data, batch_first=True, padding_value=vocab['<pad>'])
    return padded_data
padded_data_en = numericalize_and_pad(tokenized_data_en, vocab_en)
padded_data_fr = numericalize_and_pad(tokenized_data_fr, vocab_fr)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1825077 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader
dataset = torch.utils.data.TensorDataset(padded_data_en, padded_data_fr)
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

##Model for testing

In [ ]:
import torch
from torch import nn
import math
class RelativePositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, max_len: int = 5000):
        super(RelativePositionalEncoding, self).__init__()
        self.emb_size = emb_size
        self.max_len = max_len
        relative_positions = torch.arange(-max_len, max_len + 1, dtype=torch.long)
        scales = torch.exp(-torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        relative_positional_encodings = torch.zeros((2 * max_len + 1, emb_size))
        relative_positional_encodings[:, 0::2] = torch.sin(relative_positions.unsqueeze(-1) * scales)
        relative_positional_encodings[:, 1::2] = torch.cos(relative_positions.unsqueeze(-1) * scales)
        self.register_buffer('relative_positional_encodings', relative_positional_encodings)

    def forward(self, length: int):
        center_pos = self.max_len
        return self.relative_positional_encodings[center_pos - length + 1 : center_pos + 1]

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size: int):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: torch.Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

class TransformerModel(nn.Module):
    def __init__(self, num_tokens_en, num_tokens_fr, embed_size, nhead, dim_feedforward, max_seq_length):
        super(TransformerModel, self).__init__()
        self.embed_size = embed_size
        self.src_tok_emb = TokenEmbedding(num_tokens_en, embed_size)
        self.tgt_tok_emb = TokenEmbedding(num_tokens_fr, embed_size)
        self.positional_encoding = RelativePositionalEncoding(embed_size, max_len=max_seq_length)

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_size, nhead=nhead, dim_feedforward=dim_feedforward, dropout=0.1)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)

        decoder_layer = nn.TransformerDecoderLayer(d_model=embed_size, nhead=nhead, dim_feedforward=dim_feedforward, dropout=0.1)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=3)
        self.generator = nn.Linear(embed_size, num_tokens_fr)

    def encode(self, src, src_mask):
        src_emb = self.src_tok_emb(src) + self.positional_encoding(src.size(1))
        return self.transformer_encoder(src_emb, src_key_padding_mask=src_mask)

    def decode(self, tgt, memory, tgt_mask, tgt_key_padding_mask):
        tgt_emb = self.tgt_tok_emb(tgt) + self.positional_encoding(tgt.size(1))
        return self.transformer_decoder(tgt_emb, memory, tgt_mask, tgt_key_padding_mask=tgt_key_padding_mask)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None, src_padding_mask=None, tgt_padding_mask=None):
        memory = self.encode(src, src_padding_mask)
        output = self.decode(tgt, memory, tgt_mask, tgt_padding_mask)
        return self.generator(output)

    def generate_square_subsequent_mask(self, sz):
        mask = torch.triu(torch.ones(sz, sz)).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask
model_adamw = TransformerModel(len(vocab_en), len(vocab_fr), embed_size=64, nhead=8, dim_feedforward=1024, max_seq_length=512)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Adam/relative_positional_encoding/crossentropyloss/model_weights.pt'))

<All keys matched successfully>

In [ ]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()
          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  print(f'Input: {input_sentence}')
                  print(f'Target: {target_sentence}')
                  print(f'Predicted: {predicted_sentence}')
                  print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss = test_and_evaluate(model, test_dataloader, seq2seq_loss, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Input: Secondly , the problems of unemployment , which are not tackled under the Structural Funds alone , underscore the importance of efficient coordination of the implementation of funds from the various EU and national support programmes and of restructuring and allocating more money to all those funds that generate new jobs . As the rapporteur rightly emphasises , there was some progress in this area in 1998 , but it was still far from adequate . 

Target: Deuxièmement D' Ceci En Toutefois Le Monsieur L' Nous Permettez À Il L' C' C' Personne Mais Toutes Dans Les Pour En Tous Mon En Dans La Cela Les Merci J Je
Predicted: , il rapporteur de chômage , mais a se pas les fonds structurels et mais que importance de une coordination des et la Union des fonds et fonds Fonds nationaux emploi et , de , mais une plus , de efficacité plus de la les fonds structurels mais les plus il ont des la Union et je rapporteur , a dit , l' de mais importance 1998 , été , par le fonds réalisés le domaine 

In [ ]:
model_ls.load_state_dict(torch.load('/content/drive/MyDrive/Adam/relative_positional_encoding/crossentropylosswithlabelsmoothning/model_cel_state_dict.pkl'))

<All keys matched successfully>

In [ ]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss_ls = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX,label_smoothing=0.1)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ls.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()
          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  print(f'Input: {input_sentence}')
                  print(f'Target: {target_sentence}')
                  print(f'Predicted: {predicted_sentence}')
                  print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_ls = test_and_evaluate(model_ls, test_dataloader, seq2seq_loss_ls, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Input: Secondly , the problems of unemployment , which are not tackled under the Structural Funds alone , underscore the importance of efficient coordination of the implementation of funds from the various EU and national support programmes and of restructuring and allocating more money to all those funds that generate new jobs . As the rapporteur rightly emphasises , there was some progress in this area in 1998 , but it was still far from adequate . 

Target: Deuxièmement D' Ceci En Toutefois Le Monsieur L' Nous Permettez À Il L' C' C' Personne Mais Toutes Dans Les Pour En Tous Mon En Dans La Cela Les Merci J Je
Predicted: , il rapporteur de rapporteur , qui a se pas l' les fonds structurels , dans que importance de une coordination de l' UE des fonds communautaires fonds programmes nationaux emploi au , de , de une coordination , de une coordination de l' les différents structurels de les que il ont des fonds UE et le rapporteur , a été , l' de le a 1998 , été le par le fonds , le do

In [ ]:
model_n.load_state_dict(torch.load('/content/drive/MyDrive/Adam/relative_positional_encoding/sparsecategoricalcrossentropy/model_weights_sc.pkl'))

<All keys matched successfully>

In [ ]:
from torchtext.data.metrics import bleu_score
import evaluate
from torch.nn import functional as F
seq2seq_loss_n = nn.NLLLoss(ignore_index=PAD_IDX)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_n.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          logits = F.log_softmax(logits, dim=-1)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()
          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  print(f'Input: {input_sentence}')
                  print(f'Target: {target_sentence}')
                  print(f'Predicted: {predicted_sentence}')
                  print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_ls = test_and_evaluate(model_n, test_dataloader, seq2seq_loss_n, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Input: The foodstuffs legislation in the European Union is the most ambitious in the world . 

Target: La Dans Monsieur Laissez C' Il La La Nous Les Il Pourquoi Nous Selon Je Par C' Concrètement Les Nous Je Ne Le Monsieur Je Pour Peu Un Le La La Monsieur
Predicted: législation européenne est l' Union européenne est ambitieux plus ambitieuse . monde .
Predicted with repetitions: législation européenne est l' Union européenne est ambitieux plus ambitieuse . monde . 
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Input: As Mr MacCormick has pointed out , there h

In [ ]:
model_adamw.load_state_dict(torch.load('/content/drive/MyDrive/Adamw/relative_positional_encoding/crossentropyloss/model_metrics_adamw.pkl'))

<All keys matched successfully>

In [ ]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss_adamw = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_adamw.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()
          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  print(f'Input: {input_sentence}')
                  print(f'Target: {target_sentence}')
                  print(f'Predicted: {predicted_sentence}')
                  print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_adamw = test_and_evaluate(model_adamw, test_dataloader, seq2seq_loss_adamw, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Input: There is an additional recent opinion sent on 11 August last year and there is a letter of formal notice sent on 10 April this year for failure to apply a number of articles properly . 

Target: Il Monsieur Nous Nous Je Mais L' La Le Madame Nous La C' Ce À Je Doivent À Je La Le Le Il Espérons C' Comme Madame C' Dans Pour Il Monsieur
Predicted: lymphatiques PPLE lymphatiques définitive majesté opposantes reconnaîtra majesté Confrontée né majesté bouteille né exporte opposantes Parlons majesté Confrontée majesté né reconnaîtra gratuité glissant González majesté né opposantes
Predicted with repetitions: lymphatiques PPLE lymphatiques définitive majesté opposantes reconnaîtra majesté Confrontée né né majesté bouteille né né exporte opposantes Parlons majesté Confrontée majesté né né reconnaîtra gratuité gratuité glissant González majesté majesté majesté né né opposantes opposantes opposantes opposantes opposantes opposantes opposantes opposantes opposantes opposantes opposantes oppo

In [ ]:
model_adamw_ls.load_state_dict(torch.load('/content/drive/MyDrive/Adamw/relative_positional_encoding/crossentropywithlabelsmoothning/model_state_adamw_c.pkl'))

<All keys matched successfully>

In [ ]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss_adamw_ls = nn.CrossEntropyLoss(ignore_index=PAD_IDX,label_smoothing=0.5)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_adamw_ls.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()

          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  print(f'Input: {input_sentence}')
                  print(f'Target: {target_sentence}')
                  print(f'Predicted: {predicted_sentence}')
                  print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_adamw = test_and_evaluate(model_adamw_ls, test_dataloader, seq2seq_loss_adamw_ls, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Input: Before the vote : 

Target: Avant Permettez Il Le La Dès Je L La OPCVM Je ( L' L' Abandonnez Cher En Les On J' Quelle De On Quand Je Sans Un Souscrire Une Et Nous Il
Predicted: le vote sur à ,
Predicted with repetitions: le vote sur 
 à , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,

Input: Let me , in conclusion , quote Mr Prodi who said that immigration must become a factor in development . 

Target: le -moi y rapport possibilité lors voudrais ' Commission 
 perçois Le utilisation UE la collègue revanche chemins peut aimerais est plus

In [ ]:
model_adamw_n.load_state_dict(torch.load('/content/drive/MyDrive/Adamw/relative_positional_encoding/sparsecategoricalcrossentropy/model_metrics_adamw_n.pt'))

<All keys matched successfully>

In [ ]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss_adamw_n = nn.NLLLoss(ignore_index=PAD_IDX)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_adamw_n.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          logits = F.log_softmax(logits, dim=-1)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()

          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  print(f'Input: {input_sentence}')
                  print(f'Target: {target_sentence}')
                  print(f'Predicted: {predicted_sentence}')
                  print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_adamw = test_and_evaluate(model_adamw_n, test_dataloader, seq2seq_loss_adamw_n, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Input: The foodstuffs legislation in the European Union is the most ambitious in the world . 

Target: La Dans Monsieur Laissez C' Il La La Nous Les Il Pourquoi Nous Selon Je Par C' Concrètement Les Nous Je Ne Le Monsieur Je Pour Peu Un Le La La Monsieur
Predicted: situation européenne est l' Union européenne est la plus grande . monde . de est
Predicted with repetitions: situation européenne est l' Union européenne est la plus grande . monde . 
 de est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est est es

In [ ]:
model_sgd.load_state_dict(torch.load('/content/drive/MyDrive/SGD/relative_positional_encoding/crossentropyloss/model_sgd_state.pkl'))

<All keys matched successfully>

In [ ]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss_sgd = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_sgd.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()
          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  print(f'Input: {input_sentence}')
                  print(f'Target: {target_sentence}')
                  print(f'Predicted: {predicted_sentence}')
                  print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_sgd = test_and_evaluate(model_sgd, test_dataloader, seq2seq_loss_sgd, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Input: There is an additional recent opinion sent on 11 August last year and there is a letter of formal notice sent on 10 April this year for failure to apply a number of articles properly . 

Target: Il Monsieur Nous Nous Je Mais L' La Le Madame Nous La C' Ce À Je Doivent À Je La Le Le Il Espérons C' Comme Madame C' Dans Pour Il Monsieur
Predicted: , la ,
Predicted with repetitions: , , , , , , , , , , , , , , , , , , , , , , , , , , , , la , , 
 , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,

Input: Mr President , allow me to begin by thanking the rapporteur , Mr Ferber , for the good work he 

In [ ]:
model_sgd_ls.load_state_dict(torch.load('/content/drive/MyDrive/SGD/relative_positional_encoding/crossentropywithlabelsmoothning/model_sgd_ls_state.pkl'))

<All keys matched successfully>

In [ ]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss_sgd_ls = nn.CrossEntropyLoss(ignore_index=PAD_IDX, label_smoothing=0.1)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_sgd_ls.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()
          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  print(f'Input: {input_sentence}')
                  print(f'Target: {target_sentence}')
                  print(f'Predicted: {predicted_sentence}')
                  print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_sgd_ls = test_and_evaluate(model_sgd_ls, test_dataloader, seq2seq_loss_sgd_ls, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Input: There is an additional recent opinion sent on 11 August last year and there is a letter of formal notice sent on 10 April this year for failure to apply a number of articles properly . 

Target: Il Monsieur Nous Nous Je Mais L' La Le Madame Nous La C' Ce À Je Doivent À Je La Le Le Il Espérons C' Comme Madame C' Dans Pour Il Monsieur
Predicted: de , de , de , de , de , de ,
Predicted with repetitions: de de , , , de de , , , , , , , de , de de , , , , , , , , , de , , , 
 de , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,

Input: Mr President , allow me to begin by thanking the rapporteur , Mr

In [ ]:
model_sgd_n.load_state_dict(torch.load('/content/drive/MyDrive/SGD/relative_positional_encoding/sparsecategoricalcrossentropy/model_sgd_n_state.pkl'))

<All keys matched successfully>

In [ ]:
from torchtext.data.metrics import bleu_score
import evaluate
seq2seq_loss_sgd_n = nn.NLLLoss(ignore_index=PAD_IDX)
def remove_repetitions(sentence):
    tokens = sentence.split()
    previous_token = None
    filtered_tokens = []
    for token in tokens:
        if token != previous_token:
            filtered_tokens.append(token)
            previous_token = token
    return ' '.join(filtered_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_sgd_n.to(device)
def test_and_evaluate(model, test_dataloader, loss_fn, vocab_en, vocab_fr, num_examples=10):
    model.eval()
    total_test_loss = 0
    examples_printed = 0
    meteor = evaluate.load('meteor')
    references = []
    candidates = []
    with torch.no_grad():
      for src, tgt in test_dataloader:
          src, tgt = src.to(device), tgt.to(device)
          src = src.T
          tgt_inp = tgt.T
          tgt_input = tgt_inp[:-1, :]
          tgt_output = tgt_inp[1:, :]
          src_seq_length = src.shape[0]
          tgt_seq_length = tgt_input.shape[0]
          src_mask = torch.zeros(src_seq_length, src_seq_length).to(device).type(torch.bool)
          tgt_mask = model.generate_square_subsequent_mask(tgt_seq_length).to(device)
          src_padding_mask = (src == PAD_IDX).transpose(0, 1).to(device)
          tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)

          logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask)
          logits = F.log_softmax(logits, dim=-1)
          loss = loss_fn(logits.view(-1, logits.shape[-1]), tgt_output.reshape(-1))
          total_test_loss += loss.item()
          if examples_printed < num_examples:
              predictions = logits.argmax(dim=-1)
              for i in range(predictions.size(1)):
                  if examples_printed >= num_examples:
                      break
                  # input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['PAD_IDX'], vocab_en['<BOS>'], vocab_en['<EOS>'])])
                  input_sentence = ' '.join([vocab_en.get_itos()[idx] for idx in src[:, i] if idx not in (vocab_en['<pad>'], vocab_en['<bos>'], vocab_en['<eos>'])])
                  target_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in tgt[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # predicted_sentence = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  # translated
                  predicted_sentence = remove_repetitions(' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])]))
                  predicted_sentence_with_repititions = ' '.join([vocab_fr.get_itos()[idx] for idx in predictions[:, i] if idx not in (vocab_fr['<pad>'], vocab_fr['<bos>'], vocab_fr['<eos>'])])
                  print(f'Input: {input_sentence}')
                  print(f'Target: {target_sentence}')
                  print(f'Predicted: {predicted_sentence}')
                  print(f'Predicted with repetitions: {predicted_sentence_with_repititions}')
                  print()
                  examples_printed += 1
                  references.append([target_sentence])
                  candidates.append(predicted_sentence)
    meteor_score = meteor.compute(predictions=candidates, references=references)
    print(f'Meteor Score: {meteor_score}')
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss:.4f}')
    return avg_test_loss
test_loss_sgd_ls = test_and_evaluate(model_sgd_n, test_dataloader, seq2seq_loss_sgd_n, vocab_en, vocab_fr)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Input: The foodstuffs legislation in the European Union is the most ambitious in the world . 

Target: La Dans Monsieur Laissez C' Il La La Nous Les Il Pourquoi Nous Selon Je Par C' Concrètement Les Nous Je Ne Le Monsieur Je Pour Peu Un Le La La Monsieur
Predicted: , de ,
Predicted with repetitions: , , , , , , , , de , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,

Input: As Mr MacCormick has pointed out , there have been cases where Parliament has decided to waive immunity . 

Target: législation certain le -moi est est remise Commission allons États